In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [5]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [6]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {}
for i in range(len(world_set)):
  vocab[world_set[i]] = i

In [7]:
print(vocab)

{'i': 0, 'c': 1, 'r': 2, 'w': 3, 'k': 4, 'y': 5, 'b': 6, 'o': 7, 'B': 8, 'e': 9, 'l': 10, "'": 11, 's': 12, 'h': 13, 'p': 14, 'g': 15, 'n': 16, '.': 17, ',': 18, 'u': 19, 'f': 20, 't': 21, ' ': 22, 'a': 23, 'm': 24, 'd': 25}


In [8]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [9]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [10]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1: i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [17]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(type(x_data))
print(y_data[0])

<class 'list'>
[2, 0, 1, 4, 22, 3, 23, 10, 10, 12]


In [18]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor: 32비트 부동소수점, LongTesor: 64비트 정수(부호있음) 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot) #연산
Y = torch.LongTensor(y_data) #label이므로 정수

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  if __name__ == '__main__':


In [23]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [24]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0.,

In [25]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 2,  0,  1,  4, 22,  3, 23, 10, 10, 12])


In [26]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size= input_dim, hidden_size= hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, vocab_size)
  
  def forward(self, x):
    x,_ = self.rnn(x) 
    x = self.fc(x) 
    return x

In [27]:
net = Net(vocab_size, hidden_size, 2)

In [28]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [32]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [33]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, 26), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

akkpkkkkokkkkkkkkkkkkkkkkkkkkkpkkkkkkkkkkkktkkkkktkkkkkkkkkkkkkkkkkkkkkkkkkokkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkpkkkkkkkkkkkkkkkkkkkkkkkkkkkkokkkkkkkkkkkkkkkkkkkkkkkkkkkkkpkkkkkkkkkkkkkkkkkk
ak kktkttttttt tttttttttkkttt ttttttt ttttkt ktttttkttttttttttttttttk tktkttttttt tttttttttttttttttttttttttttttttttttttttttttttktttttttttttttk tktkttttttt tttttttttttttkttttttkttttkttk tttttttttt t
     t t  ttt  ttt t tt   ttt ttttttt t t tt tttttttttt ttttt ttt tt    t t  ttt  tt tttt  ttttt  tt ttt t tt tttt tt t ttttt t t tttt ttt tt    t t  ttt  ttt t ttttttt tttttt ttt   t  t ttttttt   
                 t        t   t   tt    t  t    tt   t     tt   t                 tt   t           t               t       t    t   tt   t                   t        t       t            t  t t    
                                                                                                                                                                                                     
          

In [34]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to khow us how badly we want thinks."

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [35]:
#1. 생성할 문장 데이터

sentence = ("인생의 가치는 그 길이에 있지 않다."
            "우리는 하루하루를 사용하여 인생을 만들어 나간다."
            "오래 살 수는 있지만, 그렇다고 꼭 많은 걸 얻게 되진 않는다.")

In [43]:
len(list(sentence))

82

In [36]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {}
for i in range(len(world_set)):
  vocab[world_set[i]] = i

In [37]:
print(vocab)

{'있': 0, '다': 1, '가': 2, '하': 3, '는': 4, '래': 5, '꼭': 6, '이': 7, '지': 8, '렇': 9, '용': 10, '걸': 11, '를': 12, '않': 13, '만': 14, '치': 15, '들': 16, '되': 17, '많': 18, '인': 19, '오': 20, '어': 21, '고': 22, '살': 23, '우': 24, '여': 25, '의': 26, '을': 27, '.': 28, '수': 29, '생': 30, '그': 31, ',': 32, '은': 33, '리': 34, '나': 35, ' ': 36, '사': 37, '게': 38, '진': 39, '길': 40, '루': 41, '에': 42, '간': 43, '얻': 44}


In [38]:
#3. 문자 집합 크기 확인
vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 45


In [40]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 6  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [41]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1: i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 인생의 가치 -> 생의 가치는
1 생의 가치는 -> 의 가치는 
2 의 가치는  ->  가치는 그
3  가치는 그 -> 가치는 그 
4 가치는 그  -> 치는 그 길
5 치는 그 길 -> 는 그 길이
6 는 그 길이 ->  그 길이에
7  그 길이에 -> 그 길이에 
8 그 길이에  ->  길이에 있
9  길이에 있 -> 길이에 있지
10 길이에 있지 -> 이에 있지 
11 이에 있지  -> 에 있지 않
12 에 있지 않 ->  있지 않다
13  있지 않다 -> 있지 않다.
14 있지 않다. -> 지 않다.우
15 지 않다.우 ->  않다.우리
16  않다.우리 -> 않다.우리는
17 않다.우리는 -> 다.우리는 
18 다.우리는  -> .우리는 하
19 .우리는 하 -> 우리는 하루
20 우리는 하루 -> 리는 하루하
21 리는 하루하 -> 는 하루하루
22 는 하루하루 ->  하루하루를
23  하루하루를 -> 하루하루를 
24 하루하루를  -> 루하루를 사
25 루하루를 사 -> 하루를 사용
26 하루를 사용 -> 루를 사용하
27 루를 사용하 -> 를 사용하여
28 를 사용하여 ->  사용하여 
29  사용하여  -> 사용하여 인
30 사용하여 인 -> 용하여 인생
31 용하여 인생 -> 하여 인생을
32 하여 인생을 -> 여 인생을 
33 여 인생을  ->  인생을 만
34  인생을 만 -> 인생을 만들
35 인생을 만들 -> 생을 만들어
36 생을 만들어 -> 을 만들어 
37 을 만들어  ->  만들어 나
38  만들어 나 -> 만들어 나간
39 만들어 나간 -> 들어 나간다
40 들어 나간다 -> 어 나간다.
41 어 나간다. ->  나간다.오
42  나간다.오 -> 나간다.오래
43 나간다.오래 -> 간다.오래 
44 간다.오래  -> 다.오래 살
45 다.오래 살 -> .오래 살 
46 .오래 살  -> 오래 살 수
47 오래 살 수 -> 래 살 수는
48 래 살 수는 ->  살 수는 
49  살 수는  -> 살 수는 있
50 살 수는 있 

In [44]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(type(x_data))
print(y_data[0])

<class 'list'>
[30, 26, 36, 2, 15, 4]


In [45]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor: 32비트 부동소수점, LongTesor: 64비트 정수(부호있음) 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot) #연산
Y = torch.LongTensor(y_data) #label이므로 정수

In [46]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([76, 6, 45])
레이블의 크기 : torch.Size([76, 6])


In [47]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
      

In [48]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([30, 26, 36,  2, 15,  4])


In [49]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size= input_dim, hidden_size= hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, vocab_size)
  
  def forward(self, x):
    x,_ = self.rnn(x) 
    x = self.fc(x) 
    return x

In [50]:
net = Net(vocab_size, hidden_size, 2)

In [51]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [52]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([76, 6, 45])


In [54]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, 45), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭어꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭어꼭어꼭꼭꼭꼭꼭어꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭꼭
꼭             꼭                                                                  
                                                                                 
                                                                                 
                                                                                 
                                                                                 
                                                                                 
                                                                                 
                                                                                 
                                                    는    는                       
   하   그  는  하지 그  는   하               는     는      는 그  는    는      그          는
   하   그  는  있지 그  는   있              하루  하는 는      는 있  는 그  는 그    있  는 않  않  는
   하   그 하는  있지 

In [55]:
predict_str

'생을 가치는 그 길이에 있지 않다.우리는 하루하루를 사용하여 인생을 만들어 나간다.오래 살 수는 있지만, 그렇다고 꼭 많은 걸 얻게 되진 않는다.'